Creating a messy dataset

In [4]:
import pandas as pd
import numpy as np

data = {
    'CustomerID': [1, 2, 2, 3, np.nan, 4, 5],
    'Name': [' Alice', 'bob ', 'Bob', 'Charlie', 'DAVID', ' Eve', 'Frank'],
    'Email': ['alice@example.com', 'bob@', 'bob@example.com', np.nan, 'david@example.com', 'eve@example', 'frank@example.com'],
    'Age': ['25', 30, 'thirty', 35, -99, 28, 200],
    'JoinDate': ['2023-01-15', '2023/02/20', '2023-03-10', '2023-04-05', '2023-05-12', '2023-06-30', '2023-07-01'],
    'Purchase': [150.5, 200, 200, 75.0, 120.75, 99.99, 150.5]
}
df_raw = pd.DataFrame(data)
df_raw.to_csv('messy_data.csv', index=False)

Data Cleaning

In [11]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('messy_data.csv')
original_shape = df.shape

print('Shape:', df.shape)
print('\nInfo:')
df.info()
print('\nSample')
df.head()

Shape: (7, 6)

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   CustomerID  6 non-null      float64
 1   Name        7 non-null      object 
 2   Email       6 non-null      object 
 3   Age         7 non-null      object 
 4   JoinDate    7 non-null      object 
 5   Purchase    7 non-null      float64
dtypes: float64(2), object(4)
memory usage: 468.0+ bytes

Sample


,CustomerID,Name,Email,Age,JoinDate,Purchase
0,1.0,Alice,alice@example.com,25,2023-01-15,150.50
1,2.0,bob,bob@,30,2023/02/20,200.00
2,2.0,Bob,bob@example.com,thirty,2023-03-10,200.00
3,3.0,Charlie,NaN,35,2023-04-05,75.00
4,NaN,DAVID,david@example.com,-99,2023-05-12,120.75


Handling Duplicates

In [18]:
df.drop_duplicates(inplace=True)
df.drop_duplicates(subset=['CustomerID', 'Email'], keep='first', inplace=True)

Missing values

In [19]:
print('\nMissing Count:')
print(df.isna().sum())


Missing Count:
CustomerID    1
Name          0
Email         1
Age           0
JoinDate      0
Purchase      0
dtype: int64


In [ ]:
Strategy per column(drop row(column))

In [22]:
df.dropna(subset=['CustomerID'], inplace=True)

Email: fill with placeholder if you need a value

In [23]:
df['Email'].fillna('unknown@email.com', inplace=True)

Data type conversion

In [25]:
df['Age'] = pd.to_numeric(df['Age'], errors='coerce')
df['JoinDate'] = pd.to_datetime(df['JoinDate'], errors='coerce')

Text Cleaning & Standardization

In [27]:
str_cols = df.select_dtypes(include='object').columns
for col in str_cols:
    df[col] = df[col].str.strip().str.title()

Email validation (basic)
import re

In [28]:
import re
email_pattern = re.compile(r'^[^@]+@[^@]+\.+[^@]+$')
df['Email_Valid'] = df['Email'].apply(lambda x: bool(email_pattern.match(x)))

Outlier Treatment (Age example)

In [31]:
Q1 = df['Age'].quantile(0.25)
Q3 = df['Age'].quantile(0.75)
IQR= Q3 - Q1
lower = Q1 - 1.5*Q3
upper = Q3 + 1.5*Q1

print(f'\nAge outliner (IQR): < {lower} or > {upper}')
outliners = df[(df['Age'] < lower) | (df['Age'] > upper)]

df['Age'] = df['Age'].clip(lower, upper)


Age outliner (IQR): < -24.5 or > 77.0


Final Missing Value Imputation

In [32]:
df['Age'].fillna(df['Age'].median(), inplace=True)

C:\Users\Anik\AppData\Local\Temp\ipykernel_3636\1933487976.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(), inplace=True)


Save clean data

In [33]:
df.to_csv('messy_data.csv', index=False)

Summary Report

In [46]:
print('\n' + '='*50)
print('Cleaning Reports')
print('='*50)
print(f'Originals rows/cols: {original_shape}')
print(f'Final rows/cols : {df.shape}')
print(f'Rows removed : {original_shape[0] - df.shape[0]}')
print('\nFinal dtypes:')
print(df.dtypes)
print(df.head())


Cleaning Reports
Originals rows/cols: (7, 6)
Final rows/cols : (6, 7)
Rows removed : 1

Final dtypes:
CustomerID            float64
Name                   object
Email                  object
Age                   float64
JoinDate       datetime64[ns]
Purchase              float64
Email_Valid              bool
dtype: object
   CustomerID     Name              Email   Age   JoinDate  Purchase  \
0         1.0    Alice  Alice@Example.Com  25.0 2023-01-15    150.50   
1         2.0      Bob               Bob@  30.0        NaT    200.00   
2         2.0      Bob    Bob@Example.Com  30.0 2023-03-10    200.00   
3         3.0  Charlie  Unknown@Email.Com  35.0 2023-04-05     75.00   
5         4.0      Eve        Eve@Example  28.0 2023-06-30     99.99   

   Email_Valid  
0         True  
1        False  
2         True  
3         True  
5        False  
